In [44]:
#import dependencies
import pandas as pd
import os
#import google maps
import gmaps
#import google key from the config file
from config import g_key
#import requests to make api calls
import requests
#import depedency to export gmap as html
from ipywidgets.embed import embed_minimal_html

In [51]:
#configure google maps
gmaps.configure(api_key=g_key)

In [18]:
#load the city weather data which has rainfall and snowfall details from csv file
file_to_load = os.path.join("weather_data", "WeatherPy_challenge.csv")
city_weather_data_df = pd.read_csv(file_to_load)

In [19]:
def prompt_user_and_validate(textMessageToPrompt, validValuesList):
    prompt_pref = input(textMessageToPrompt)
    continueLoop = True
    while continueLoop:
        if (prompt_pref in validValuesList):
            continueLoop = False     
        else:
            print (f"Invalid data, enter either {validValuesList}")
            prompt_pref = input(textMessageToPrompt)
    return prompt_pref

In [35]:
#prompt for minimum & maximum temperature 
min_temp = float(input("What is the minimum temperature you would like for your trip ?"))
max_temp = float(input("What is the maximum temperature you would like for your trip ?"))

yes_no = ['yes','no']
#prompt for rain & snow preference
rain_pref = prompt_user_and_validate("Do you want it to be raining? (yes/no)", yes_no)
snow_pref = prompt_user_and_validate("Do you want it to be snowing? (yes/no)", yes_no )


What is the minimum temperature you would like for your trip ?55
What is the maximum temperature you would like for your trip ?70
Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)no


In [36]:
#filter the city weather data based on temperature preference
preferred_cities_df = city_weather_data_df[(city_weather_data_df['Max Temp'] >= min_temp) & 
                                             (city_weather_data_df['Max Temp'] <= max_temp)]

print (f" Original number of cities : {city_weather_data_df['City'].count()}")
print (f"preferred number of cities: {preferred_cities_df['City'].count()}")

 Original number of cities : 574
preferred number of cities: 112


In [37]:
#filter the city weather data based on snow or rain preference
if (rain_pref == 'yes'):
    preferred_cities_df = preferred_cities_df[(preferred_cities_df['Rain inches (last 3 hrs)'] > 0)]

if (snow_pref == 'yes'):
    preferred_cities_df = preferred_cities_df[(preferred_cities_df['Snow inches (last 3 hrs)'] > 0)]

print (f"Total number of cities based on preference: {preferred_cities_df['City'].count()}")

Total number of cities based on preference: 112


In [38]:
#create a copy of the preferred cities with few of the columns for populating hotels data
hotels_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotels_df["Hotel Name"] = ""

hotels_df.head(13)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Lincoln,AU,64.60,clear sky,-34.73,135.87,
8,Ushuaia,AR,69.80,scattered clouds,-54.80,-68.30,
14,Bluff,NZ,55.99,light rain,-46.60,168.33,
26,Cape Town,ZA,66.99,few clouds,-33.93,18.42,
31,Busselton,AU,66.99,clear sky,-33.65,115.33,
34,Itoman,JP,64.40,scattered clouds,26.12,127.67,
50,Barcelos,PT,59.00,clear sky,41.54,-8.62,
52,Zhuhai,CN,64.99,clear sky,22.28,113.57,
56,Port Alfred,ZA,64.99,light rain,-33.59,26.89,
58,Safaga,EG,60.80,clear sky,26.73,33.94,


In [28]:
def find_and_populate_nearest_hotel(preferred_cities_hotels_df):
    #parameters for search for a hotel
    params = {
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }

    #base url for getting nearest hotel from google api using longitude and lattitude
    nearbysearch_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #iterate over lattitude and longitude data
    for index, row in hotels_df.iterrows():
        latitude = row["Lat"]
        longitude = row["Lng"]

        #add longitude and lattitude to params
        params["location"] = f"{latitude},{longitude}"

        hotels_data = requests.get(nearbysearch_base_url, params=params).json()

        #get the name of the nearest first hotel
        try:
            preferred_cities_hotels_df.loc[index, "Hotel Name"] = hotels_data["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... Skipping")

    preferred_cities_hotels_df.head(5)

In [40]:
find_and_populate_nearest_hotel(hotels_df)
hotels_df.head(20)

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Lincoln,AU,64.60,clear sky,-34.73,135.87,Port Lincoln Hotel
8,Ushuaia,AR,69.80,scattered clouds,-54.80,-68.30,Albatros Hotel
14,Bluff,NZ,55.99,light rain,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
26,Cape Town,ZA,66.99,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
31,Busselton,AU,66.99,clear sky,-33.65,115.33,Observatory Guest House
34,Itoman,JP,64.40,scattered clouds,26.12,127.67,Southern Beach Hotel & Resort Okinawa
50,Barcelos,PT,59.00,clear sky,41.54,-8.62,Hotel Do Terço
52,Zhuhai,CN,64.99,clear sky,22.28,113.57,珠海粤财假日酒店
56,Port Alfred,ZA,64.99,light rain,-33.59,26.89,The Halyards Hotel
58,Safaga,EG,60.80,clear sky,26.73,33.94,Lamar Resort Abu Soma


In [41]:
#save the hotels dataframe as csv
output_file = "weather_data/WeatherPy_vacation.csv"
hotels_df.to_csv(output_file)

In [42]:
def plot_gmap_with_markers(data_tobe_plotted_df):
    
    #create infoblock template whose values will be replace iterating
    info_block_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp}\xb0F</dd>
    </dl>
    """
    #create infoblock for display
    hotel_info = [info_block_template.format(**row) for index, row in data_tobe_plotted_df.iterrows()]

    #create heatmap from hotels dataframe for location and temperatures
    locations = data_tobe_plotted_df[["Lat", "Lng"]]
    temperatures = data_tobe_plotted_df["Max Temp"]

    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.85)
    heat_layer = gmaps.heatmap_layer(locations, weights=temperatures, dissipating=False, max_intensity=300, point_radius=4)

    marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

    fig.add_layer(heat_layer)
    fig.add_layer(marker_layer)
    #export as html
    embed_minimal_html('weather_data/preferred_cities_with_hotel.html', views=[fig])
    #show the figure
    fig

In [52]:
#create infoblock template whose values will be replace iterating
info_block_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}\xb0F</dd>
</dl>
"""
#create infoblock for display
hotel_info = [info_block_template.format(**row) for index, row in hotels_df.iterrows()]

#create heatmap from hotels dataframe for location and temperatures
locations = hotels_df[["Lat", "Lng"]]
temperatures = hotels_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.85)
heat_layer = gmaps.heatmap_layer(locations, weights=temperatures, dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#export as html
#fig.save('weather_data/WeatherPy_vacation_map.png')
embed_minimal_html('weather_data/preferred_cities_with_hotel.html', views=[fig])
#show the figure
fig

Figure(layout=FigureLayout(height='420px'))